Visualize Prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import math

def draw_box(ax, x, y, z, w, l, h, yaw, color='r'):
    corners = np.array([
        [w/2, l/2, -h/2],
        [-w/2, l/2, -h/2],
        [-w/2, -l/2, -h/2],
        [w/2, -l/2, -h/2],
        [w/2, l/2, h/2],
        [-w/2, l/2, h/2],
        [-w/2, -l/2, h/2],
        [w/2, -l/2, h/2],
    ])


    R = np.array([
        [math.cos(yaw), -math.sin(yaw), 0],
        [math.sin(yaw),  math.cos(yaw), 0],
        [0, 0, 1]
    ])
    corners = (R @ corners.T).T + np.array([x, y, z])

    faces = [
        [corners[i] for i in [0,1,2,3]], # bottom
        [corners[i] for i in [4,5,6,7]], # top
        [corners[i] for i in [0,1,5,4]], # front
        [corners[i] for i in [2,3,7,6]], # back
        [corners[i] for i in [1,2,6,5]], # left
        [corners[i] for i in [0,3,7,4]], # right
    ]

    ax.add_collection3d(Poly3DCollection(faces, facecolors=color, linewidths=0.5, edgecolors='k', alpha=0.3))

frame_id = "20250428_1318_1319_000719"
points = np.load(f"data/custom/points/{frame_id}.npy")
x, y, z = points[:, 0].clip(-80, 10), points[:, 1].clip(-10, 80), points[:, 2].clip(-5, 10)

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, s=0.1, c="gray")

gt = f"data/custom/labels/{frame_id}.txt"
pred = f"prediction/{frame_id}.txt"
with open(pred, "r") as f:
    for line in f:
        vals = line.strip().split()
        x, y, z = map(float, vals[0:3])
        w, l, h = map(float, vals[3:6])
        yaw = float(vals[6])
        label = vals[7]
        color = 'red' if label == 'Truck' else 'blue'
        if len(vals) == 9:
            score = float(vals[8])
            if score > 0.5:
                draw_box(ax, x, y, z, w, l, h, yaw, color)
        else:
            draw_box(ax, x, y, z, w, l, h, yaw, color)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Point Cloud + 3D Boxes')
ax.set_box_aspect([1, 1, 0.1])
ax.view_init(elev=90, azim=0)
plt.tight_layout()
plt.show()



Prediction pkl to Label txt format

In [ ]:

import pickle as pkl
file_path = "output/dsvt_models/dsvt_plain_1f_onestage_SL/default/eval/epoch_60/val/default/result.pkl"
with open(file_path, "rb") as f:
    results = pkl.load(f)
# name, score, boxes_lidar, pred_labels, frame_id

for i, pred in enumerate(results):
    coord = pred['boxes_lidar']
    class_name = pred['name']
    conf = pred['score']
    file_path = f"./prediction_val/{pred['frame_id']}.txt"
    with open (file_path, "w") as f:
        for j in range(len(class_name)):
            x, y, z, dx, dy, dz, yaw = coord[j]
            name = class_name[j]
            score = conf[j]
            line = f"{x:.2f} {y:.2f} {z:.2f} {dx:.2f} {dy:.2f} {dz:.2f} {yaw:.2f} {name} {score:.4f}\n"
            f.write(line)


Label txt to JSON for SL visualizer

In [ ]:
import os
import json
from glob import glob

txt_dir = "prediction_1"
pcd_prefix = "/SL_Lidar_Dataset"
output_path = "output_2.json"

frames = []

txt_files = sorted(glob(os.path.join(txt_dir, "*.txt")))

for txt_file in txt_files:
    basename = os.path.basename(txt_file).replace(".txt", "")
    frame_id = basename.split("_")[-1]
    frame_date = "_".join(basename.split("_")[:-1])
    pcd_name = f"merged_frame_{frame_id}.pcd"
    folder_name = basename.split("_")[0][2:]
    pcd_file = os.path.join(pcd_prefix, folder_name, frame_date, "merged_pcds", pcd_name)
    track_id = 1
    with open(txt_file, "r") as f:
        objects = []
        for line in f:
            values = line.strip().split()
            if float(values[-1]) < 0.5:
                continue
            x, y, z, dx, dy, dz, yaw = map(float, values[:7])
            class_name = values[-2]
            if class_name == "Truck":
                class_id = 1
            elif class_name == "Forklift":
                class_id = 2
            elif class_name == "Worker":
                class_id = 3
            else:
                print(f"{class_name}")
                assert False

            obj = {
                "x_center": x,
                "y_center": y,
                "z_center": z,
                "width": dx,
                "height": dy,
                "length": dz,
                "yaw": yaw,
                "class_id": int(class_id),
                "track_id": int(track_id),
                "attribute": int(0)
            }
            objects.append(obj)
            track_id += 1
    
    frame = {
        "file_name": pcd_file,
        "timestamp": "0000-00-00T00:00:00",  # 필요 시 수정
        "objects": objects
    }
    frames.append(frame)

with open(output_path, "w") as f:
    json.dump({"frames": frames}, f, indent=4)

print(f"Saved to {output_path}")
